In [7]:
# Basic setup
%run ./config.ipynb

Cortex Python SDK v5.4.2


In [10]:
from cortex import Message
from langdetect import detect_langs


def detect(params):
    msg = Message(params)
    text = msg.payload.get('text')

    results = detect_langs(text)

    others = []
    for lang in results[1:]:
        others.append({'lang': lang.lang, 'score': lang.prob})

    top_lang = results[0]

    return Message.with_payload(
        {'text': text, 'lang': top_lang.lang, 'score': top_lang.prob, 'other_langs': others}).to_params()

In [12]:
import sys
import argparse
from pprint import pprint


def test_detect(args):
    text = u'¡Bienvenido a CognitiveScale, que tengas un buen día!'
    print('Detecting language in: {}'.format(text))
    m = Message.with_payload({'text': text})
    result = detect(m.to_params())
    pprint(result['payload'])

test_detect(None)

Detecting language in: ¡Bienvenido a CognitiveScale, que tengas un buen día!
{'lang': 'es',
 'other_langs': [],
 'score': 0.999996917941852,
 'text': '¡Bienvenido a CognitiveScale, que tengas un buen día!'}


In [22]:
import sys
from cortex import Cortex

_shim = """
def main(params):
    return %s(params)
"""


def load_source(path, fn):
    with open(path) as f:
        source = f.read()
    return '{}\n\n{}'.format(source, _shim % fn)


def build_detect_action(builder):
    action = builder.action('c12e/lang-detect').from_source(
        load_source('lang_detect/detect.py', 'detect')).with_requirements(['langdetect']).daemon().build()

    return action


def build_detect_skill(builder, action):
    b = builder.skill('c12e/lang-detect-skill').title('Language Detection').description(
        'Detects the language used in a provided text.')

    b.input('text_input').parameter('text', 'string', None, 'Text').all_routing(
        action, 'text_with_language').build()

    b.output('text_with_language').parameter('text', 'string', None, 'Text').parameter(
        'lang', 'string', None, 'Language').parameter('score', 'number', 'float', 'Score').parameter(
        'other_langs', 'array', None, 'Other Languages').build()

    return b.build()


def build_detect():
    cortex = Cortex.client()
    builder = cortex.builder()

    action = build_detect_action(builder)
    print('Deployed action {} at version v{}'.format(action.name, action.version))
    print(action.get_deployment_status())

    skill = build_detect_skill(builder, action)
    print('Deployed skill {} at version v{}'.format(skill.name, skill.version))

build_detect()


Building Cortex Action (daemon): c12e/lang-detect
Building Docker image private-registry.cortex.insights.ai/jja-test/c12e_lang-detect:ic0a7nj...
Step 1/13 : FROM continuumio/miniconda3:4.5.4
Step 2/13 : WORKDIR /app
Removing intermediate container c9cb9e88c198
Step 3/13 : RUN apt-get update && apt-get install -y linux-headers-amd64 build-essential
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://security.debian.org/debian-security stretch/updates InRelease [94.3 kB]
Get:3 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:4 http://deb.debian.org/debian stretch Release [118 kB]
Get:5 http://deb.debian.org/debian stretch Release.gpg [2434 B]
Get:6 http://security.debian.org/debian-security stretch/updates/main amd64 Packages [566 kB]
Get:7 http://deb.debian.org/debian stretch-updates/main amd64 Packages [5476 B]
Get:8 http://deb.debian.org/debian stretch/main amd64 Packages [9500 kB]
Fetched 10.4 MB in 2s (3819 kB/s)
Reading package lists...
Reading pac